In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFECV

pd.set_option('display.max_columns', 27)

In [2]:
#Loading data
data1=pd.read_csv("data1.csv",sep=';')
data1.drop('ID', axis=1, inplace=True)
data1_details=pd.read_csv("data1_details.csv",sep=';')
data_1=pd.concat([data1_details, data1],join_axes=[data1_details.index],axis=1)
data_1.set_index('ID', inplace=True)

data2=pd.read_csv("data2.csv",sep=';')
data2.drop('ID', axis=1, inplace=True)
data2_details=pd.read_csv("data2_details.csv",sep=';')
data_2=pd.concat([data2_details, data2],join_axes=[data2_details.index],axis=1)
data_2.set_index('ID', inplace=True)

data3=pd.read_csv("data3.csv",sep=';')
data3.drop('ID', axis=1, inplace=True)
data3_details=pd.read_csv("data3_details.csv",sep=';')
data_3=pd.concat([data3_details, data3],join_axes=[data3_details.index],axis=1)
data_3.set_index('ID', inplace=True)

data4=pd.read_csv("data4.csv",sep=';')
data4.drop('ID', axis=1, inplace=True)
data4_details=pd.read_csv("data4_details.csv",sep=';')
data_4=pd.concat([data4_details, data4],join_axes=[data4_details.index],axis=1)
data_4.set_index('ID', inplace=True)

dataset=data_1.append([data_2,data_3,data_4])
cols = list(dataset.columns.values)
cols
dataset=dataset[['UserID','Domain','TopDomain','cname','ip','Adult Content','Bias','Commercial','Confidence','Database','Date','Discussion','Educational/Research',
                 'Hosting Type','Language','Media','Neutrality','News/Editorial','Other Problem','Personal/Leisure',
                 'Readability-Lang','Readability-Vis','Trustiness','Web Spam','auto_lang',
                  'nowwwID','wwwID']]
dataset.head()

,UserID,Domain,TopDomain,cname,ip,Adult Content,Bias,Commercial,Confidence,Database,Date,Discussion,Educational/Research,Hosting Type,Language,Media,Neutrality,News/Editorial,Other Problem,Personal/Leisure,Readability-Lang,Readability-Vis,Trustiness,Web Spam,auto_lang,nowwwID,wwwID
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,
10221,21.0,ch.transco.eu,transco.eu,ch.transco.eu,129.143.40.61,NonAdult,2.0,Commercial,Sure,NonDatabase,26-MAR-10 14:11:34,NonDiscussion,NonEducational,HTNormal,de,NonMedia,2.0,NonNews-Edit,NoOtherProblem,NonPersonal-Leisure,2.0,2.0,2.0,NonSpam,de,NaN,NaN
5154,21.0,bimaq.eu,bimaq.eu,bimaq.eu,134.102.10.129,NonAdult,2.0,NonCommercial,Sure,NonDatabase,26-MAR-10 13:10:01,NonDiscussion,Educational,HTNormal,de,NonMedia,2.0,NonNews-Edit,NoOtherProblem,NonPersonal-Leisure,2.0,2.0,2.0,NonSpam,de,NaN,NaN
59769,20.0,austrom.eu,austrom.eu,austrom.eu,193.186.185.72,NaN,NaN,NaN,NaN,NaN,26-MAR-10 14:57:25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OtherProblem,NaN,NaN,NaN,NaN,NaN,de,NaN,NaN
74416,22.0,anderquelle.eu,anderquelle.eu,anderquelle.eu,193.25.115.91,NonAdult,NaN,NonCommercial,Sure,NonDatabase,26-MAR-10 16:02:41,NonDiscussion,NonEducational,HTNormal,de,NonMedia,NaN,NonNews-Edit,NoOtherProblem,Personal-Leisure,2.0,2.0,NaN,NonSpam,de,NaN,NaN
15183,20.0,cartoflex.eu,cartoflex.eu,cartoflex.eu,194.145.226.45,NonAdult,2.0,Commercial,Sure,NonDatabase,26-MAR-10 14:43:45,NonDiscussion,NonEducational,HTNormal,de,NonMedia,3.0,NonNews-Edit,NoOtherProblem,NonPersonal-Leisure,2.0,2.0,3.0,NonSpam,de,NaN,NaN


In [3]:
#Cleaning data
dataset=dataset.dropna()
dataset.head()

,UserID,Domain,TopDomain,cname,ip,Adult Content,Bias,Commercial,Confidence,Database,Date,Discussion,Educational/Research,Hosting Type,Language,Media,Neutrality,News/Editorial,Other Problem,Personal/Leisure,Readability-Lang,Readability-Vis,Trustiness,Web Spam,auto_lang,nowwwID,wwwID
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,
82588,14.0,www.contacts2contracts.eu,contacts2contracts.eu,www.contacts2contracts.eu,217.64.127.80,NonAdult,2.0,NonCommercial,Sure,NonDatabase,23-APR-10 18:09:33,NonDiscussion,Educational,HTNormal,en,NonMedia,3.0,NonNews-Edit,NoOtherProblem,NonPersonal-Leisure,2.0,2.0,3.0,NonSpam,en,74852.0,82588.0
33102,14.0,www.eiscltd.eu,eiscltd.eu,www.eiscltd.eu,217.64.127.80,NonAdult,2.0,NonCommercial,Sure,NonDatabase,23-APR-10 15:09:53,NonDiscussion,Educational,HTNormal,en,NonMedia,3.0,NonNews-Edit,NoOtherProblem,NonPersonal-Leisure,2.0,2.0,3.0,NonSpam,en,35432.0,33102.0
33252,14.0,www.fecma.eu,fecma.eu,fecma.eu,213.175.222.230,NonAdult,2.0,NonCommercial,Sure,NonDatabase,13-APR-10 13:00:08,NonDiscussion,Educational,HTNormal,en,NonMedia,3.0,NonNews-Edit,NoOtherProblem,NonPersonal-Leisure,2.0,2.0,3.0,NonSpam,en,35621.0,33252.0
90115,8.0,empirion.eu,empirion.eu,empirion.eu,150.254.238.3,NonAdult,2.0,NonCommercial,Sure,NonDatabase,08-APR-10 12:28:53,NonDiscussion,Educational,HTNormal,en,NonMedia,3.0,NonNews-Edit,NoOtherProblem,NonPersonal-Leisure,2.0,2.0,3.0,NonSpam,en,90115.0,58145.0
58145,8.0,www.empirion.eu,empirion.eu,empirion.eu,150.254.238.3,NonAdult,2.0,NonCommercial,Sure,NonDatabase,15-APR-10 12:21:33,NonDiscussion,Educational,HTNormal,en,NonMedia,3.0,NonNews-Edit,NoOtherProblem,NonPersonal-Leisure,2.0,2.0,3.0,NonSpam,en,90115.0,58145.0


In [4]:
                            ####    Support Vector Machines   ####

In [5]:
#Feature Enginnering
data_dummies = pd.get_dummies(dataset)
#print("Features after get_dummies:\n", list(data_dummies.columns))
display(data_dummies.head())

,UserID,Bias,Neutrality,Readability-Lang,Readability-Vis,Trustiness,nowwwID,wwwID,Domain_100-tagu.eu,Domain_1yoga.eu,Domain_2getthere.eu,Domain_4x4net.eu,Domain_abcproject.eu,...,Educational/Research_Educational,Educational/Research_NonEducational,Hosting Type_HTNormal,Language_en,Media_Media,Media_NonMedia,News/Editorial_News-Edit,News/Editorial_NonNews-Edit,Other Problem_NoOtherProblem,Personal/Leisure_NonPersonal-Leisure,Personal/Leisure_Personal-Leisure,Web Spam_NonSpam,auto_lang_en
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,
82588,14.0,2.0,3.0,2.0,2.0,3.0,74852.0,82588.0,0,0,0,0,0,...,1,0,1,1,0,1,0,1,1,1,0,1,1
33102,14.0,2.0,3.0,2.0,2.0,3.0,35432.0,33102.0,0,0,0,0,0,...,1,0,1,1,0,1,0,1,1,1,0,1,1
33252,14.0,2.0,3.0,2.0,2.0,3.0,35621.0,33252.0,0,0,0,0,0,...,1,0,1,1,0,1,0,1,1,1,0,1,1
90115,8.0,2.0,3.0,2.0,2.0,3.0,90115.0,58145.0,0,0,0,0,0,...,1,0,1,1,0,1,0,1,1,1,0,1,1
58145,8.0,2.0,3.0,2.0,2.0,3.0,90115.0,58145.0,0,0,0,0,0,...,1,0,1,1,0,1,0,1,1,1,0,1,1


In [6]:
svm1 = SVC()
array=data_dummies.values
X = array[:,0:4971]
y = data_dummies['Media_Media'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
svm1.fit(X_train, y_train)
print("Test score: {:.2f}".format(svm1.score(X_test, y_test)))

Test score: 0.99


In [7]:
svm2 = SVC(kernel='linear')
array=data_dummies.values
X = array[:,0:4971]
y = data_dummies['Media_Media'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
svm2.fit(X_train, y_train)
print("Test score: {:.2f}".format(svm2.score(X_test, y_test)))

Test score: 0.99


In [8]:
                                #Automatic Feature Selection
#Univariant Statistics
rng = np.random.RandomState(42)
noise = rng.normal(size=(len(data_dummies.values), 50))
X_w_noise = np.hstack([data_dummies.values, noise])

X_train, X_test, y_train, y_test = train_test_split(X_w_noise, data_dummies['Media_Media'] , random_state=0, test_size=.5)

select = SelectPercentile(percentile=50)
select.fit(X_train, y_train)
X_train_selected = select.transform(X_train)

print("X_train.shape: {}".format(X_train.shape))
print("X_train_selected.shape: {}".format(X_train_selected.shape))

X_test_selected = select.transform(X_test)
svm1.fit(X_train, y_train)
print("Score with all features: {:.3f}".format(svm1.score(X_test, y_test)))
svm1.fit(X_train_selected, y_train)
print("Score with only selected features: {:.3f}".format(svm1.score(X_test_selected, y_test)))

d:\python27\lib\site-packages\sklearn\feature_selection\univariate_selection.py:113: UserWarning: Features [   9   11   13 ... 4966 4969 4970] are constant.
  UserWarning)
d:\python27\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
d:\python27\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


X_train.shape: (610, 5021)
X_train_selected.shape: (610, 2510)
Score with all features: 0.990
Score with only selected features: 0.990


In [9]:
print("X_train.shape: {}".format(X_train.shape))
print("X_train_selected.shape: {}".format(X_train_selected.shape))
svm2.fit(X_train, y_train)
print("Score with all features: {:.3f}".format(svm2.score(X_test, y_test)))
svm2.fit(X_train_selected, y_train)
print("Score with only selected features: {:.3f}".format(svm2.score(X_test_selected, y_test)))

X_train.shape: (610, 5021)
X_train_selected.shape: (610, 2510)
Score with all features: 0.985
Score with only selected features: 0.990


In [10]:
#Model based feature selection
select = SelectFromModel(SVC(kernel='linear'),threshold="median")
select.fit(X_train, y_train)
X_train_l1 = select.transform(X_train)
print("X_train.shape: {}".format(X_train.shape))
print("X_train_l1.shape: {}".format(X_train_l1.shape))

X_test_l1 = select.transform(X_test)
score = SVC().fit(X_train_l1, y_train).score(X_test_l1, y_test)
print("Test score: {:.3f}".format(score))

X_train.shape: (610, 5021)
X_train_l1.shape: (610, 5021)
Test score: 0.990


In [ ]:
#Iterative feature selection
select = RFE(SVC(kernel='linear'),n_features_to_select=100)
select.fit(X_train,y_train)
X_train_rfe = select.transform(X_train)
X_test_rfe = select.transform(X_test)

score = SVC().fit(X_train_rfe, y_train).score(X_test_rfe, y_test)
print("Test score: {:.3f}".format(score))

In [ ]:
#RFE Cross Validation
rfecv = RFECV(estimator=svm2, step=1, cv=4, scoring='accuracy')
rfecv.fit(X, y)
print("Optimal number of features : %d" % rfecv.n_features_) 
print("Score::",rfecv.score(X,y))